In [1]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
import $profile.`hadoop-2.6`
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
import $ivy.`org.apache.spark::spark-mllib:2.1.0`
import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._

import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

val sparkSession = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
                    .jupyter()
                    .master("local[*]")
                    .appName("notebook")
                    .getOrCreate()
val sc = sparkSession.sparkContext

log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


import $exclude.$                        , $ivy.$                            // for cleaner logs

import $profile.$           

import $ivy.$                                   // adjust spark version - spark >= 2.0

import $ivy.$                                    

import $ivy.$                                   

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)


import org.apache.spark._

import org.apache.spark.sql._

import jupyter.spark.session._


import org.apache.spark.graphx._

import org.apache.spark.rdd.RDD


sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@57c8b69f
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@141f0d18

In [2]:
// Create an RDD for the vertices
val users: RDD[(VertexId, (String, String))] =
  sc.parallelize(Array((3L, ("rxin", "student")), (7L, ("jgonzal", "postdoc")),
                       (5L, ("franklin", "prof")), (2L, ("istoica", "prof"))))
// Create an RDD for edges
val relationships: RDD[Edge[String]] =
  sc.parallelize(Array(Edge(3L, 7L, "collab"),    Edge(5L, 3L, "advisor"),
                       Edge(2L, 5L, "colleague"), Edge(5L, 7L, "pi")))
// Define a default user in case there are relationship with missing user
val defaultUser = ("John Doe", "Missing")
// Build the initial Graph
val graph = Graph(users, relationships, defaultUser)

users: RDD[(VertexId, (String, String))] = ParallelCollectionRDD[0] at parallelize at cmd1.sc:2
relationships: RDD[Edge[String]] = ParallelCollectionRDD[1] at parallelize at cmd1.sc:6
defaultUser: (String, String) = ("John Doe", "Missing")
graph: Graph[(String, String), String] = org.apache.spark.graphx.impl.GraphImpl@4fe6700a

In [3]:
// Count all users which are postdocs
graph.vertices.filter { case (id, (name, pos)) => pos == "postdoc" }.count
// Count all the edges where src > dst
graph.edges.filter(e => e.srcId > e.dstId).count
graph.edges.filter { case Edge(src, dst, prop) => src > dst }.count

res2_0: Long = 1L
res2_1: Long = 1L
res2_2: Long = 1L

In [4]:
// Use the triplets view to create an RDD of facts.
val facts: RDD[String] = graph.triplets.map(triplet =>
    triplet.srcAttr._1 + " is the " + triplet.attr + " of " + triplet.dstAttr._1)
facts.collect.foreach(println(_))

rxin is the collab of jgonzal
franklin is the advisor of rxin
istoica is the colleague of franklin
franklin is the pi of jgonzal


facts: RDD[String] = MapPartitionsRDD[24] at map at cmd3.sc:1